## OpenVINO optimizations for Text classification task


## Import the packages needed for successful execution

In [ ]:
from transformers import AutoConfig, AutoTokenizer, default_data_collator
from datasets import load_dataset, load_metric
from optimum.intel.openvino import OVAutoModelForSequenceClassification

from torch.utils.data import DataLoader

from tqdm import tqdm

from pathlib import Path

### Instructions on conversion to OpenVINO
We will use the OpenVINO™ Integration with Optimum module to convert the quantized text classification model to an OpenVINO model object. <br>
For details on quantizing the model using NNCF, refer to `quantization/README.md` <md>
We will then use Huggingface datasets and metric to evaluate the converted model.

In [ ]:
model_dir = 'optimized_model'
base_model_name = "optimized_model/ov_model.xml"
config = AutoConfig.from_pretrained('sentence-transformers/roberta-large-nli-stsb-mean-tokens')
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/roberta-large-nli-stsb-mean-tokens')
ov_model = OVAutoModelForSequenceClassification.from_pretrained(model_dir, config=config, num_labels=1, from_ov=True)

In [ ]:
def preprocess_function(examples):
    result =  tokenizer(examples['sentence1'], examples['sentence2'], padding="max_length", max_length=128, truncation=True)
    result["labels"] = examples["similarity_score"]
    return result

dataset = load_dataset("stsb_multi_mt", name="en")
metric = load_metric('glue', 'stsb')

dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)
validation_dataset = dataset['test']
val_dataloader = DataLoader(
       validation_dataset, shuffle=True, collate_fn=default_data_collator
    )

for idx, batch in enumerate(tqdm(val_dataloader, 'Looping over the test dataset')):
    outputs_auto = ov_model(input_ids=batch['input_ids'].numpy(), attention_mask=batch['attention_mask'].numpy())
    predictions_ov_auto = outputs_auto[0]
    references = batch['labels'].numpy()
        
    metric.add_batch(predictions=[predictions_ov_auto], references=[references])

score = metric.compute()
print(f'Metric score : {score}')

### Benchmark the converted model using the benchmark app
The OpenVINO toolkit provides a benchmarking application to gauge the platform specific runtime performance that can be obtained under optimal configuration parameters for a given model. For more details refer to: https://docs.openvino.ai/latest/openvino_inference_engine_tools_benchmark_tool_README.html

In [ ]:
print('Benchmark OpenVINO model using the benchmark app')
! benchmark_app -m "$base_model_name" -d CPU -api async -t 10 -hint latency